## Proyecto Final Métodos Analíticos
### Francisco Bahena, Cristian Challu, Daniel Sharp

Carga de librerías

In [1]:
import numpy as np
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
import sys
import time
from IPython.display import clear_output
from io import StringIO

Carga de los datos y presentación:

In [2]:
import requests
import pandas as pd
flujo = pd.read_csv('flujo1.csv',header=None)
flujo.columns = ['nodo','mac','pot','fecha']

Nuestros datos consisten de un log de conexiones a nodos de 3 centros comerciales. Entre los 3 centros tenemos 32 nodos 'wifi' a los que intentan conectarse dispositivos de los individuos que transitan cerca o dentro de los centros comerciales. En los datos contamos con 4 columnas, son:  
- MAC address del Nodo de Wifi
- MAC address del dispositivo
- Potencia de la conexión
- Timestamp de la conexión

In [ ]:
flujo.head()

In [3]:
flujo['fecha'] = pd.to_datetime(flujo['fecha'], format = '%Y-%m-%d %H:%M:%S')
flujo['day'] = flujo['fecha'].apply(lambda x: x.day)
flujo['hour'] = flujo['fecha'].apply(lambda x: x.hour)
flujo['minute'] = flujo['fecha'].apply(lambda x: x.minute)

Limpiamos nuestros datos y creamos variables para día, hora y minuto de tal forma que sea más sencillo filtrar y trabajar con los datos.

In [ ]:
flujo_f = flujo.loc[(flujo['day'] == 16 )& (flujo['hour'] == 8)]

In [ ]:
flujo_f.shape

### Ejecución del Bloom filter  
Para la ejecución del bloom se 'levantó' un API en un servidor con nuestra implementación del Bloom Filter. Para simular el stream de nuestros datos, enviamos las observaciones al API en bloques de un minuto y obtenemos el número de 'nuevos' dispositivos contados por el Filtro de Bloom al igual que el número de dispositivos que ya existían. Hacemos lo mismo con una base de datos para obtener los valores reales. También obtenemos el tiempo de ejecución tanto para nuestro Filtro de Bloom como de la base de datos.  
  
Con el siguiente comando reestablecemos la base y el Filtro de Bloom. Además, enviamos parámetros para la construcción del Filtro, como en número de hashes y el tamaño del vector filtro.

In [ ]:
requests.get('http://184.72.111.50:3000/limpia_db_bloom/20/1042043')

A continuación 'simulamos' el stream de datos, enviando las observaciones en bloques de un minuto por un tiempo total de 10 horas.

In [ ]:
ultimo = 99
history = np.empty((0,5),float)
num_unicos = 0
hora = 8
hora_fin = 18
plt.close()
while hora < hora_fin:
    actual = time.localtime().tm_min
    contador = -1
    flujo_f = flujo.loc[(flujo['day'] == 16 )& (flujo['hour'] == hora)]
    hora = hora+1
    # Iteramos en cada minuto
    for minuto in range(60):
        contador += 1
        print("Hora: {} Minuto: {}".format(hora-1, minuto))
        base = flujo_f.loc[flujo_f['minute'] == minuto]
        base = base[['mac']].values.tolist()
        elementos = ['-'.join(elemento) for elemento in base]
        ultimo = actual
        # Llamamos al API y enviamos los elementos del minuto actual al Bloom Filter
        r_bloom = requests.post('http://184.72.111.50:3000/insert_elements_bloom/', json= {'records':elementos})
        #r_bloom = requests.post('http://localhost:5000/insert_elements_bloom/', json= {'records':elementos})
        elapsed_bloom = float(r_bloom.json()['tiempo_en_segundos'])
        # Llamamos al API y enviamos los elementos del minuto actual a la Base de Datos
        r_db = requests.post('http://184.72.111.50:3000/insert_elements_db/', json= {'records':elementos})
        #r_db = requests.post('http://localhost:5000/insert_elements_db/', json= {'records':elementos})
        elapsed_db = float(r_db.json()['tiempo_en_segundos'])
        tasa_error = r_db.json()['nuevas_visitas_base'] - r_bloom.json()['nuevas_visitas']
        num_unicos += r_bloom.json()['nuevas_visitas']
        
        step = [contador, elapsed_bloom, elapsed_db, tasa_error, num_unicos]
        history = np.vstack((history, step))
        
        # Graficamos nuestros resultados para las diferentes métricas consideradas
        clear_output()
        print(r_bloom.json())
        print(r_db.json())
        plt.figure(1)
        plt.figure(figsize = (15, 5))
        plt.subplot(131)
        plt.plot(history[:,1], label = "Bloom", linestyle = '-')
        plt.plot(history[:,2], label = "DataBase", linestyle = '--')
        plt.legend()
        plt.xlabel("Minutos")
        plt.ylabel("Tiempo de ejecución")
        plt.title("Tiempos ejecución")
        plt.xlim((1,600))
        plt.subplot(132)
        plt.plot(history[:,3], linestyle = '-')
        plt.ylabel("Tasa error")
        plt.xlabel("Minutos")
        plt.title("Tasa de error")
        plt.xlim((1,600))
        plt.subplot(133)
        plt.plot(history[:,4], linestyle = '-')
        plt.ylabel("Número de únicos")
        plt.xlabel("Minutos")
        plt.title("Número de únicos")
        plt.xlim((1,600))
        plt.show()
    #time.sleep(1)
    #print(time.localtime())

El Bloom Filter nos permite ver el número de individuos únicos en cierto periodo de tiempo al igual que detectar, en un periodo de un minuto de tiempo, cuantos de los dispositivos son nuevos y cuantos ya habían aparecido en alguno de los 3 centros comerciales.

### Hyperloglog


In [17]:
test = flujo.mac.values.tolist()

In [23]:
requests.get('http://localhost:5000/limpia_hloglog/20')

<Response [200]>

In [24]:
r_hll = requests.post('http://localhost:5000/check_unique/', json= {'records':test})

In [25]:
r_hll.json()

{'unicas hloglog': 525821.8441022076, 'unicas_base': 611162}